# CDNow消费行为数据分析-task10

首先看下这个数据集，4个特征：ID，购买日期，订单数，订单金额

首先导入基本的库

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime

这四个字段处理啊上有个好处，都是数字，没有字符，感觉分析上可能也方便些

In [ ]:
df = pd.read_table('CDNOW.txt',names = ['ID','Date','Count','Price'],sep = '\s+')

In [ ]:
df.head()

In [ ]:
df.describe()

通过对数据基本的分析，可以看出OrderPirce的最大值是1286.01，最小值是0，平均值是36，可以有个基本认识，价格高的商品不是很多，因为平均价格时36左右，那是不是要考虑下1286.01的商品价格有没有可能数据有问题

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df['Date'] = pd.to_datetime(df.Date, format='%Y%m%d')

In [ ]:
df['Month'] = df.Date.values.astype('datetime64[M]')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

依然没空值，好开心

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
plt.figure(figsize=(20,15))

In [ ]:
plt.subplot(221)
df.groupby('Month')['Price'].sum().plot(fontsize=24) 
plt.title('TotalPrice',fontsize=24) 


In [ ]:
plt.subplot(222)
df.groupby('Month')['Date'].count().plot(fontsize=24)
plt.title('PayNum',fontsize=24) 



In [ ]:
plt.subplot(223)
df.groupby('Month')['Count'].sum().plot(fontsize=24)
plt.title('TotalPrice',fontsize=24)


In [ ]:
plt.subplot(224)
df.groupby('Month')['ID'].apply(lambda x:len(x.unique())).plot(fontsize=24)
plt.title('PersonNum',fontsize=24)
plt.tight_layout()
plt.show()

In [ ]:
person = df.groupby('ID').sum()

In [ ]:
person.describe()

In [ ]:
person.query('Price < 500').plot.scatter(x='Price',y='Count')

In [ ]:
person.Price.plot.hist(20)

In [ ]:
person.query("Price< 800")['Price'].plot.hist(bins=20)

In [ ]:
person.query('Count < 100').Count.hist(bins = 40)

In [ ]:
order_diff = df.groupby('ID').apply(lambda x:x['Date'] - x['Date'].shift())


In [ ]:
order_diff.head(10)

In [ ]:
order_diff.describe()

In [ ]:
plt.figure(figsize=(15,5))
plt.hist((order_diff / np.timedelta64(1, 'D')).dropna(), bins = 50)
plt.xlabel('消费周期',fontsize=24)
plt.ylabel('频数',fontsize=24)
plt.title('用户消费周期分布图',fontsize=24);

In [ ]:
orderdt_min=df.groupby('ID').Date.min()
orderdt_max=df.groupby('ID').Date.max()
(orderdt_max-orderdt_min).head()

In [ ]:
(orderdt_max-orderdt_min).mean()

In [ ]:
((orderdt_max-orderdt_min)/np.timedelta64(1,'D')).hist(bins=15)

In [ ]:
life_time = (orderdt_max - orderdt_min).reset_index()

In [ ]:
life_time.head()

In [ ]:
plt.figure(figsize=(10,5))
life_time['life_time'] = life_time.Date / np.timedelta64(1,'D')
life_time[life_time.life_time > 0].life_time.hist(bins = 100, figsize = (12,6))

In [ ]:
life_time[life_time.life_time>0].Date.mean()

In [ ]:
a = df.groupby('ID')['Date'].agg(['min','max']).reset_index()
new_old = (a['min'] == a['max']).value_counts().values

plt.pie(x = new_old,
       autopct = '%.1f%%',
       shadow = True,
       explode = [0.08,0],
       textprops = {'fontsize' : 11})
plt.axis('equal') 
plt.legend(['one','mult'])

In [ ]:
pivoted_df=df.pivot_table(index='ID',columns='Month',values='Date',
                       aggfunc='count').fillna(0)
pivoted_df.head()
pivoted_df_transf=pivoted_df.applymap(lambda x: 1 if x>1 else np.nan if x==0 else 0)
pivoted_df_transf.head()

In [ ]:
df_duplicate =pd.DataFrame(pivoted_df_transf.sum()/pivoted_df_transf.count()).reset_index()
df_duplicate.columns = ['Date', 'DuplicatedRate']
df_duplicate['Date'] = df_duplicate.Date.astype(str).apply(lambda x:x[:-3])

plt.figure(figsize = (15,6))
plt.plot(df_duplicate.Date, df_duplicate.DuplicatedRate)
plt.xlabel('time', fontsize=24)
plt.ylabel('rebuy',fontsize=24)
# plt.ylim(0,1)
plt.title('change',fontsize=24)

In [ ]:
pivoted_money=df.pivot_table(index='ID',columns='Date',values='Price',
                             aggfunc='mean').fillna(0)



In [ ]:
columns_month=df.Month.sort_values().astype('str').unique()


In [ ]:
columns_month

In [ ]:
pivoted_money.columns=columns_month


In [ ]:
pivoted_money.head()

In [ ]:
df.head()

In [ ]:
df_purchase = (pivoted_purchase_return.sum() / pivoted_purchase_return.count()).reset_index()
df_purchase.columns = ['Date', 'PurchaseRate']
df_purchase['Date'] = df_purchase.Date.astype(str).apply(lambda x:x[:-3])

plt.figure(figsize = (15,5))
plt.plot(df_purchase.Date, df_purchase.PurchaseRate)
plt.xlabel('时间', fontsize=24)
plt.ylabel('回购率', fontsize=24)
plt.title('回购率的变化', fontsize=24);